# urbex

In [ ]:
# noqa

from pathlib import Path
import subprocess
import geopandas as gpd
import pandas as pd
import numpy as np
from whitebox.whitebox_tools import WhiteboxTools
import matplotlib.pyplot as plt
import matplotlib as mpl
import os
import leafmap.leafmap as lm
import rasterio
import sys

pd.options.mode.copy_on_write = True

# Add parent directory to path

module_path = str(Path(os.getcwd()).parents[1])
if module_path not in sys.path:
    sys.path.append(module_path)
os.chdir(module_path)

In [ ]:
# noqa

from IO.Scripts.Modules.utilities import create_dd_con, move_pathlib  # noqa
from IO.Scripts.Modules.setup import (  # noqa
    dwnld_import,
    create_extents,
    wgs84_to_utm,
    folder_set_up,
    test_set_up,
    alt_poly,
)
from IO.Scripts.Modules.terrain import get_terrain_data  # noqa
from IO.Scripts.Modules.vectors import (  # noqa
    ox_trans_load,
    filt_load_bldgs,
    water_dwnld_clean,
    points_2_csv,
    roads_dwnld_filt,
    places_dwnld,
    big_roads_only,
)
from IO.Scripts.Modules.rasters import (  # noqa
    run_kde,
    kde_to_city_center,
    export_kde_raster,
    vec_2_rast,
    distance_accumulation,
    node_dist_grid,
    array_2_tif,
    background_points,
    stack_rasters,
    samples_with_data,
    tif_2_ascii,
    raster_out,
)
from IO.Scripts.Modules.netx import node2cc  # noqa

## Intro (Always Run)

In [ ]:
# INPUTS
cityid = 597
out_path = Path(r"C:\Users\MAtkinson\Documents\GitHub\urbex\IO\Data")
maxent = r"C:\Users\MAtkinson\Documents\GitHub\urbex\IO\maxent\maxent.jar"
release = "2025-12-17.0"
overture_location = "s3://overturemaps-us-west-2"
alt = {
    "fp": out_path / r"Rwanda/Kigali/Google/wb_KK_polygons/Kigali_Kamonyi.shp",
    "id": "597000",
    "city": "Kigali",
    "country": "Rwanda",
}  # or None

In [ ]:
# Download city shape, name, and country
if isinstance(alt, dict):
    uc = alt_poly(fp=alt["fp"], id=alt["id"], city=alt["city"], country=alt["country"])
else:
    uc = dwnld_import(cityid, out_path)

# Create extent data and projected crs from city shape
extentPoly, zoom, bounds, xmin, xmax, ymin, ymax = create_extents(uc.geometry)
UTMZone, wkid = wgs84_to_utm(uc.geometry)

# extract city and country name
city_name = uc["GC_UCN_MAI_2025"].tolist()[0]
country_name = uc["GC_CNT_GAD_2025"].tolist()[0]

# set up folder structure
outfp, fdict = folder_set_up(out_path, city_name, country_name)
test_set_up()
print(outfp)

## DOWNLOAD DATA

In [ ]:
# download data

# connect to duckdb
con = create_dd_con()

# elevation
get_terrain_data(bounds, zoom, fdict["Downloads"], city_name)

# transportation
ox_trans_load(extentPoly, fdict["Downloads"], city_name, delfile=False)

# buildings
filt_load_bldgs(
    dloc=f"{overture_location}/release/{release}/theme=buildings/type=building/*",
    big_out=None,
    pt_out=None,
    samp_out=str(fdict["Downloads"] / f"{city_name}_bldg_samp_pts.shp"),
    cols="subtype, class, height, num_floors, geometry",
    xmin=xmin,
    xmax=xmax,
    ymin=ymin,
    ymax=ymax,
    con=con,
)

# roads
roads_dwnld_filt(
    fdict,
    city_name,
    xmin,
    xmax,
    ymin,
    ymax,
    oloc=overture_location,
    relnum=release,
    con=con,
    delfile=False,
)

# places
places_dwnld(
    fdict,
    city_name,
    xmin,
    xmax,
    ymin,
    ymax,
    oloc=overture_location,
    relnum=release,
    con=con,
    delfile=False,
)

# water
water_dwnld_clean(
    city_name,
    fdict["Intermediate"],
    xmin,
    xmax,
    ymin,
    ymax,
    wkid,
    oloc=overture_location,
    relnum=release,
    con=con,
    delfile=False,
)

## DATA PREP FOR MAXENT

#### Kernel Density  Estimation of City Centers

Sources:
- https://jakevdp.github.io/blog/2013/12/01/kernel-density-estimation/
- https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KernelDensity.html#sklearn.neighbors.KernelDensity
- https://www.spatialanalysisonline.com/HTML/index.html?density__kernels_and_occupancy.htm
- https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-analyst/how-kernel-density-works.htm
- https://gistbok-topics.ucgis.org/AM-03-008
- https://pygis.io/docs/e_summarize_vector.html#method-2-display-and-export-with-scikit-learn

In [ ]:
plFPath = fdict["Downloads"] / f"{city_name}_plcs.shp"
places = gpd.read_file(plFPath)

In [ ]:
kde_places, xct, yct = run_kde(places, xmin, ymin, xmax, ymax, 0.005)

In [ ]:
cc_pt = kde_to_city_center(
    kde_places, xct, yct, city_name, country_name, xmin, xmax, ymin, ymax
)

In [ ]:
# Plot data
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
cc_pt.plot(ax=ax, markersize=1, color="yellow")
ax.imshow(np.rot90(kde_places), cmap="Reds", extent=(xmin, xmax, ymin, ymax))
uc.plot(ax=ax, color="none", edgecolor="dimgray")
ax.set_title(
    f"{city_name}, {country_name} - Kernel Density Estimation for Places",
    fontdict={"fontsize": "15", "fontweight": "3"},
)
plt.show()

In [ ]:
# Export raster
fp = fdict["Intermediate"] / f"{city_name}_plc_kde.tif"
export_kde_raster(kde_places, xct, yct, xmin, xmax, ymin, ymax, proj=4326, filename=fp)
# Export City Center
cc_pt.to_file(fdict["Intermediate"] / f"{city_name}_city_center.shp")

#### Get Distance to City Center

Sources:
 - https://docs.momepy.org/en/stable/user_guide/graph/convert.html
 - https://gdal.org/en/stable/tutorials/gdal_grid_tut.html
 - https://github.com/nathanjmcdougall/geospatial-wheels-index

In [ ]:
# read in roads and city center
rdFPath = fdict["Intermediate"] / f"{city_name}_con_rds.shp"
filt_rds = gpd.read_file(rdFPath)

ccFPath = fdict["Intermediate"] / f"{city_name}_city_center.shp"
cc = gpd.read_file(ccFPath).to_crs(wkid)

# get distance from nodes to city center
nodes = node2cc(filt_rds, cc, wkid).to_crs(4326)
nodes.to_file(fdict["Intermediate"] / f"{city_name}_nodes_dist.shp")

del filt_rds, cc

In [ ]:
# turn nodes with distance into grid w/ idw
grid_z, grid_x, grid_y, xy_coords = node_dist_grid(nodes, xmin, xmax, ymin, ymax)

In [ ]:
# plot distance grid
plt.plot(xy_coords[:, 0], xy_coords[:, 1], "k.", ms=1)
plt.imshow(grid_z.T, extent=(0, 1, 0, 1), origin="lower")
plt.colorbar()
plt.title("Cubic")
plt.show()

In [ ]:
# turn grid array to tiff file (export)
idw_fp = array_2_tif(grid_z, grid_x, grid_y, fdict, city_name, xmin, xmax, ymin, ymax)

del idw_fp, grid_z, grid_x, grid_y

### Slope

In [ ]:
elevationPath = f"{city_name}_elevation.tif"
slopePath = f"{city_name}_slope.tif"

In [ ]:
# if first time it will download the extension
# even if already in uv environment
wbt = WhiteboxTools()
wbt.set_working_dir(fdict["Downloads"])
wbt.slope(elevationPath, slopePath, None, units="degrees")

In [ ]:
for fp in [elevationPath, slopePath]:
    if not Path(fdict["Model_Inputs"] / fp).exists():
        move_pathlib(
            src_path=(fdict["Downloads"] / fp), dest_path=(fdict["Model_Inputs"] / fp)
        )
    else:
        print("Please delete file first before replacing.")

### Distance Accumulation

In [ ]:
slopeTIF = fdict["Model_Inputs"] / f"{city_name}_slope.tif"

In [ ]:
# distance accumulation from roads
roadPath = fdict["Downloads"] / f"{city_name}_rds.shp"
roadTIF = fdict["Intermediate"] / f"{city_name}_rds.tif"
roaddistTIF = fdict["Model_Inputs"] / f"{city_name}_rd_dist.tif"

filtRdPath = big_roads_only(roadPath, fdict, city_name)
vec_2_rast(filtRdPath, slopeTIF, roadTIF)
distance_accumulation(roadTIF, roaddistTIF)

In [ ]:
# distance accumulation from water
waterPath = fdict["Intermediate"] / f"{city_name}_all_water.shp"
waterTIF = fdict["Intermediate"] / f"{city_name}_all_water.tif"
waterdistTIF = fdict["Model_Inputs"] / f"{city_name}_all_water_dist.tif"

vec_2_rast(waterPath, slopeTIF, waterTIF)
distance_accumulation(waterTIF, waterdistTIF)

In [ ]:
# distance accumulation from transit
transPath = fdict["Downloads"] / f"{city_name}_trans.shp"
transTIF = fdict["Intermediate"] / f"{city_name}_trans.tif"
transdistTIF = fdict["Model_Inputs"] / f"{city_name}_trans_dist.tif"

vec_2_rast(transPath, slopeTIF, transTIF)
distance_accumulation(transTIF, transdistTIF)

## CREATE MAXENT INPUTS

#### Raster Prep

In [ ]:
rename_dict = {
    f"{city_name}_elevation": "elevation",
    f"{city_name}_all_water_dist": "wt_dist",
    f"{city_name}_nodes2cc_dist_idw_coreg": "cc_dist",
    f"{city_name}_rd_dist": "rd_dist",
    f"{city_name}_slope": "slope",
    f"{city_name}_trans_dist": "tr_dist",
}

In [ ]:
# tif to ascii
for f in Path(fdict["Model_Inputs"]).glob("*.tif"):
    new_stem = rename_dict[f.stem]
    tif_2_ascii(f, fdict["Model_Inputs_ASCII"] / f"{new_stem}.asc")

In [ ]:
rdict = {
    "elevation": fdict["Model_Inputs"] / f"{city_name}_elevation.tif",
    "wt_dist": fdict["Model_Inputs"] / f"{city_name}_all_water_dist.tif",
    "cc_dist": fdict["Model_Inputs"] / f"{city_name}_nodes2cc_dist_idw_coreg.tif",
    "rd_dist": fdict["Model_Inputs"] / f"{city_name}_rd_dist.tif",
    "slope": fdict["Model_Inputs"] / f"{city_name}_slope.tif",
    "tr_dist": fdict["Model_Inputs"] / f"{city_name}_trans_dist.tif",
}
rout = fdict["Downloads"] / f"{city_name}_rstack.tif"

In [ ]:
rnames = stack_rasters(rdict, rout)

#### Background Points

In [ ]:
sample_raster = fdict["Model_Inputs"] / f"{city_name}_elevation.tif"
bp_shp = fdict["Intermediate"] / f"{city_name}_background_points.shp"

In [ ]:
bps = background_points(
    sample_raster,
    bp_shp,
)

#### Samples With Data

In [ ]:
samp = fdict["Downloads"] / f"{city_name}_bldg_samp_pts.shp"
s_swd_shp = fdict["Presence_Data"] / f"{city_name}_bldg_samp_swd.shp"
s_swd_csv = fdict["Presence_Data"] / f"{city_name}_bldg_samp_swd.csv"
b_swd_shp = fdict["Presence_Data"] / f"{city_name}_background_swd.shp"
b_swd_csv = fdict["Presence_Data"] / f"{city_name}_background_swd.csv"

In [ ]:
samples_with_data(samples=samp, lnames=rnames, rstack_fp=rout, s_out_fp=s_swd_shp)
samples_with_data(samples=bp_shp, lnames=rnames, rstack_fp=rout, s_out_fp=b_swd_shp)

In [ ]:
points_2_csv(s_swd_shp, s_swd_csv, swd=True, layers=rnames, delfile=True)
points_2_csv(b_swd_shp, b_swd_csv, swd=True, layers=rnames, delfile=True)

## MAXENT

#### Raster Inputs to MAXENT:
- Distance to City Center
- Distance from Roads
- Distance from Water
- Distance from Transit
- Slope
- Elevation

#### Sources:
- [GitHub Repo for MAXENT](https://github.com/mrmaxent/Maxent)
- [Download & Documentation Page for MAXENT](https://biodiversityinformatics.amnh.org/open_source/maxent/)

Instructions:
Download MAXENT zip from AMNH and unzip it into folder named "MAXENT" in the IO folder. Make sure you have the "maxent.jar" file. Also make sure you have Java downloaded on your computer.

#### Run MAXENT

In [ ]:
subprocess.call(
    [
        "java",
        "-jar",
        str(maxent),
        f"environmentallayers={b_swd_csv}",
        f"projectionlayers={fdict['Model_Inputs_ASCII'] }",
        f"samplesfile={s_swd_csv}",
        f"outputdirectory={fdict['Model_Outputs']}",
        "redoifexists",
        "autorun",
    ]
)

In [ ]:
prj = (
    'GEOGCS["GCS_WGS_1984",'
    'DATUM["D_WGS_1984",'
    'SPHEROID["WGS_1984",6378137.0,298.257223563]],'
    'PRIMEM["Greenwich",0.0],'
    'UNIT["Degree",0.0174532925199433]'
    "]"
)
for x in fdict["Model_Outputs"].glob("*.asc"):
    with open(
        Path(fdict["Model_Outputs"], f"{x.stem}.prj"), "w", encoding="utf-8"
    ) as q:
        q.write(prj)
    print()

#### Convert ASCII to TIFF

In [ ]:
ascfp = fdict["Model_Outputs"] / "Building_Model_Inputs_ASCII.asc"
exfp = fdict["Model_Inputs"] / f"{city_name}_elevation.tif"
outfp = fdict["Model_Outputs"] / "Building_Model_Inputs_ASCII.tif"

In [ ]:
raster_out(rast=rasterio.open(ascfp).read(1), ofp=outfp, exfp=exfp)

## DISPLAY MAXENT OUTPUT

In [ ]:
m = lm.Map(center=(extentPoly.centroid.y, extentPoly.centroid.x), zoom=12)
m.add_basemap("Esri.WorldImagery")
m.add_raster(
    str(outfp), colormap=mpl.colormaps["turbo"], layer_name="urbex output"
)  # noqa

In [ ]:
# m